<a href="https://colab.research.google.com/github/sarahlorenzen/Outamation-Extern/blob/main/3_2_PyMuPDF_Regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PyMuPDF (if not already installed)
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.3 MB/s eta 0:00:00


In [2]:
# Import Python's built-in regular expression (Regex) module

import re

# Step 1: Identify Key Information from the Loan Worksheet

Step 1: Identify Key Information from the Loan Worksheet

1.   Applicant Name
2.   Loan Program
3.   Loan Amount
4.   Interest Rate
5.   Loan Term
6.   Total Monthly Payment







# Step 2: Use PyMuPDF to Extract the Data

In [3]:
# Upload the PDF
from google.colab import files
uploaded = files.upload()


Saving LenderFeesWorksheetNew.pdf to LenderFeesWorksheetNew.pdf


In [4]:
# Import PyMuPDF and extract data
import fitz  # PyMuPDF

## Extract Text

In [8]:
# Load the PDF
doc = fitz.open("LenderFeesWorksheetNew.pdf")

# Print metadata
print("PDF Metadata:", doc.metadata)

PDF Metadata: {'format': 'PDF 1.3', 'title': 'POINT Document', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Amyuni PDF Converter version 4.5.2.3', 'creationDate': "D:20151001155359-07'00'", 'modDate': '', 'trapped': '', 'encryption': None}


In [9]:
# Extract text from the first page
page = doc[0]
print(page.get_text())

Your actual rate, payment, and cost could be higher. Get an official Loan Estimate before choosing a loan.
Fee Details and Summary
Applicants:
Application No:
Date Prepared:
Loan Program:
Prepared By:
THIS IS NOT A GOOD FAITH ESTIMATE (GFE). This "Fees Worksheet" is provided for informational purposes ONLY, to assist
you in determining an estimate of cash that may be required to close and an estimate of your proposed monthly mortgage 
payment. Actual charges may be more or less, and your transaction may not involve a fee for every item listed.
Total Loan Amount:  
Interest Rate:
Term/Due In:
Fee
Paid To
Paid By (Fee Split**)
Amount
PFC / F / POC
TOTAL ESTIMATED FUNDS NEEDED TO CLOSE:
TOTAL ESTIMATED MONTHLY PAYMENT:
Total Estimated Funds
Total Monthly Payment
Purchase Price (+)
Alterations (+)
Land (+)
Refi (incl. debts to be paid off) (+)
Est. Prepaid Items/Reserves (+)
Est. Closing Costs (+)
Loan Amount (-)
Principal & Interest
Other Financing (P & I)
Hazard Insurance
Real Estate Tax

In [10]:
# Extract structured text as blocks
blocks = page.get_text("blocks", sort=True)

# Print each block
for block in blocks:
    print(f"Block: {block}", "\n")

Block: (36.0, 26.097490310668945, 587.3409423828125, 38.3197021484375, 'Your actual rate, payment, and cost could be higher. Get an official Loan Estimate before choosing a loan.\n', 0, 0) 

Block: (237.60000610351562, 42.08164596557617, 368.0303649902344, 57.68054962158203, 'FEES WORKSHEET\n', 12, 0) 

Block: (230.39999389648438, 58.18072509765625, 376.87939453125, 71.53446197509766, 'Fee Details and Summary\n', 1, 0) 

Block: (39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094, 'Applicants:\nApplication No:\nDate Prepared:\nLoan Program:\nPrepared By:\n', 2, 0) 

Block: (100.80000305175781, 78.44451904296875, 571.6966552734375, 107.53496551513672, 'John Q. Smith / Mary A. Smith\nsamplesmith\n10/05/2015\n30 YEAR FIXED -Purchase\nXYZ Lender\n', 13, 0) 

Block: (39.599998474121094, 119.17501068115234, 572.3463134765625, 147.94798278808594, 'THIS IS NOT A GOOD FAITH ESTIMATE (GFE). This "Fees Worksheet" is provided for informational purposes ONLY, to assist\nyo

## Extract Table Data

In [11]:
def extract_tables_with_bbox(pdf_path):
    doc = fitz.open(pdf_path)
    results = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        tables = page.find_tables()

        for i, table in enumerate(tables.tables):
            table_bbox = table.bbox
            table_data = []
            cell_bboxes = []

            for row_num in range(table.row_count):
                row_data = []
                row_bboxes = []

                for col_num in range(table.col_count):
                    cell_idx = row_num * table.col_count + col_num

                    if cell_idx < len(table.cells) and table.cells[cell_idx] is not None:
                        cell_bbox = table.cells[cell_idx]
                        cell_text = page.get_text("text", clip=cell_bbox).strip()
                        row_data.append(cell_text)
                        row_bboxes.append(cell_bbox)
                    else:
                        row_data.append("")
                        row_bboxes.append(None)

                table_data.append(row_data)
                cell_bboxes.append(row_bboxes)

            results.append({
                'page': page_num,
                'table_index': i,
                'bbox': table_bbox,
                'data': table_data,
                'cell_bboxes': cell_bboxes
            })

    doc.close()
    return results


In [12]:
# Call the function on the uploaded file
pdf_filename = list(uploaded.keys())[0]
results = extract_tables_with_bbox(pdf_filename)


In [13]:
# Print extracted data
for table in results:
    print(f"\n📄 Page {table['page']}, Table {table['table_index']}")
    for row in table['data']:
        print(row)


📄 Page 0, Table 0
['Underwriting Fee\nXYZ Lender\nBorrower\n$\n550.00', 'Wire Transfer Fee\nXYZ Lender\nBorrower\n$\n75.00']
['Administration Fee\nXYZ Lender\nBorrower\n$\n445.00', '']
['', '']

📄 Page 0, Table 1
['Appraisal Fee\nXYZ Lender\nBorrower\n$\n525.00', 'Credit Report Fee\nXYZ Lender\nBorrower\n$\n25.00']
['Tax Service Fee\nXYZ Lender\nBorrower\n$\n80.00', 'Flood Certification Fee\nXYZ Lender\nBorrower\n$\n20.00']
['', '']
['', '']

📄 Page 0, Table 2
['Closing/Escrow Fee\nSettlement Agent\nBorrower\n$\n860.00', 'Document Preparation Fee\nSettlement Agent\nBorrower\n$\n75.00']
['Notary Fee\nNOTARY\nBorrower\n$\n150.00', "Lender's Title Insurance\nBorrower\n$\n650.00"]
['Title - Courier Fee\nSettlement Agent\nBorrower\n$\n50.00', 'Electronic Document Delivery FeeSettlement Agent\nBorrower\n$\n50.00']
['', '']
['', '']
['', '']

📄 Page 0, Table 3
['Daily Interest Charges\nXYZ Lender\nBorrower\n$ 44.8611 x 25 day(s)\n$\n1,121.53', '']
